In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import json

# TO DO

- Fix functions, combine cleaning with processing, simplify.
- Make processing and harmonizing file as module

In [113]:
CURR_PATH = os.getcwd()
TARGET_PATH_SMHI = os.path.join(CURR_PATH, 'smhi_data')
TARGET_PATH_MET = os.path.join(CURR_PATH, 'met_data')

In [114]:
locations = [
    {'lat': 59.3, 'lon': 18.0},  # Stockholm
    {'lat': 55.6, 'lon': 12.5}  # Copenhagen
]

In [115]:
def _create_smhi_url(lat, lon ):
    return f"https://opendata-download-metfcst.smhi.se/api/category/pmp3g/version/2/geotype/point/lon/{lon}/lat/{lat}/data.json"


def _create_met_url(lat, lon):
    return f'https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}'

def get_smhi_data(lat, lon):
    smhi_url = _create_smhi_url(lat, lon)
    smhi_response = requests.get(smhi_url)
    
    if smhi_response.status_code == 200:
        smhi_data = smhi_response.json()
        print('SMHI data retrieved successfully')
        return smhi_data
    else:
        print(f"Failed to retrieve SMHI data: HTTP {smhi_response.status_code}")
        return None

def get_met_data(lat, lon):
    met_url = _create_met_url(lat, lon)
    headers = {'User-Agent': 'your-email@example.com'}  # Ensure you add a User-Agent
    met_response = requests.get(met_url, headers=headers)

    if met_response.status_code == 200:
        met_data = met_response.json()
        print('MET data retrieved successfully')
        return met_data
    else:
        print(f"Failed to retrieve MET data: HTTP {met_response.status_code}")
        return None

In [116]:
def fetch_api_data():
    for location in locations:
        lat = location['lat']
        lon = location['lon']
        
        smhi_data = get_smhi_data(lat, lon)
        if smhi_data:
            with open(TARGET_PATH_SMHI, 'w') as f:
                json.dump(smhi_data, f)

        met_data = get_met_data(lat, lon)
        if met_data:
            with open(TARGET_PATH_MET, 'w') as f:
                json.dump(met_data, f)

    return smhi_data, met_data

In [127]:
def process_smhi_weather_data(weather_data):
    
    time_series = weather_data['timeSeries']

    processed_data = []

    for time_entry in time_series:
        valid_time = time_entry['validTime']

        parameters = time_entry['parameters']

        processed_entry = {'valid_time': valid_time}

        for parameter in parameters:
            name = parameter['name']
            values = parameter['values'] # List of values
            values = tuple(parameter['values']) # Convert to tuple
            values = values[0]
            

            processed_entry[name] = values

        processed_data.append(processed_entry)

    smhi_df = pd.DataFrame(processed_data)

    return smhi_df

def process_met_weather_data(weather_data):

    met_df = pd.json_normalize(weather_data['properties']['timeseries'])

    met_df['valid_time'] = pd.to_datetime(met_df['time'])
    met_df = met_df.drop(columns=['time'])

    met_df = met_df.set_index('valid_time')

   
    return met_df

In [118]:
smhi_data, met_data = fetch_api_data() # DONT RUN EVERYTIME

SMHI data retrieved successfully
MET data retrieved successfully
SMHI data retrieved successfully
MET data retrieved successfully


In [128]:
met_df = process_met_weather_data(met_data)
smhi_df = process_smhi_weather_data(smhi_data)

In [129]:
def clean_met_data(met_df):

    met_df = met_df.drop(columns=['data.next_12_hours.summary.symbol_code',
                                'data.next_1_hours.summary.symbol_code',
                                'data.next_1_hours.details.precipitation_amount',
                                'data.next_6_hours.summary.symbol_code',
                                'data.next_6_hours.details.precipitation_amount'])
    
    return met_df

def clean_smhi_data(smhi_df):
    weather_mapping = {
    1: 'Clear sky',
    2: 'Nearly clear sky',
    3: 'Variable cloudiness',
    4: 'Halfclear sky',
    5: 'Cloudy sky',
    6: 'Overcast',
    7: 'Fog',
    8: 'Light rain showers',
    9: 'Moderate rain showers',
    10: 'Heavy rain showers',
    11: 'Thunderstorm',
    12: 'Light sleet showers',
    13: 'Moderate sleet showers',
    14: 'Heavy sleet showers',
    15: 'Light snow showers',
    16: 'Moderate snow showers',
    17: 'Heavy snow showers',
    18: 'Light rain',
    19: 'Moderate rain',
    20: 'Heavy rain',
    21: 'Thunder',
    22: 'Light sleet',
    23: 'Moderate sleet',
    24: 'Heavy sleet',
    25: 'Light snowfall',
    26: 'Moderate snowfall',
    27: 'Heavy snowfall'
}

    smhi_df['weather_description'] = smhi_df['Wsymb2'].map(weather_mapping)

    smhi_df = smhi_df.drop(columns=['Wsymb2', 'Wsymb2'])

    smhi_df = smhi_df.rename(columns={
        't': 'temperature',
        'vis': 'visibility',
        'wd': 'wind_direction',
        'ws': 'wind_speed',
        'r': 'humidity',
        'tstm': 'thunderstorm_probability',
        'tcc_mean': 'total_cloud_cover',
        'lcc_mean': 'low_level_cloud_cover',
        'mcc_mean': 'medium_level_cloud_cover',
        'hcc_mean': 'high_level_cloud_cover',
        'gust': 'wind_gust_speed',
        'pmin': 'min_precipitation_intensity',
        'pmax': 'max_precipitation_intensity',
        'spp': 'precipitation_frozen_form',
        'pcat': 'precipitation_category',
        'pmean': 'mean_precipitation_intensity',
        'pmedian': 'median_precipitation_intensity'
    })

    smhi_df['valid_time'] = pd.to_datetime(smhi_df['valid_time'])
    smhi_df = smhi_df.set_index('valid_time')

    return smhi_df


In [130]:
met_df = clean_met_data(met_df)
smhi_df = clean_smhi_data(smhi_df)

In [123]:
# Process data for each location
smhi_dfs = []
met_dfs = []

for location in locations:
    # lat = location['lat']
    # lon = location['lon']
    # smhi_df = process_smhi_weather_data(smhi_data)
    # met_df = process_met_weather_data(met_data)
    smhi_dfs.append(smhi_df)
    met_dfs.append(met_df)

# Concatenate all dataframes
smhi_combined_df = pd.concat(smhi_dfs, ignore_index=True)
met_combined_df = pd.concat(met_dfs, ignore_index=True)

# Merge the datasets
# merged_df = pd.merge(smhi_combined_df, met_combined_df, on=[''], suffixes=('_smhi', '_met'))

In [125]:
smhi_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   precipitation_frozen_form       156 non-null    int64  
 1   precipitation_category          156 non-null    int64  
 2   min_precipitation_intensity     156 non-null    float64
 3   mean_precipitation_intensity    156 non-null    float64
 4   max_precipitation_intensity     156 non-null    float64
 5   median_precipitation_intensity  156 non-null    float64
 6   total_cloud_cover               156 non-null    int64  
 7   low_level_cloud_cover           156 non-null    int64  
 8   medium_level_cloud_cover        156 non-null    int64  
 9   high_level_cloud_cover          156 non-null    int64  
 10  temperature                     156 non-null    float64
 11  msl                             156 non-null    float64
 12  visibility                      156 

In [131]:
smhi_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 78 entries, 2024-08-19 22:00:00+00:00 to 2024-08-29 12:00:00+00:00
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   precipitation_frozen_form       78 non-null     int64  
 1   precipitation_category          78 non-null     int64  
 2   min_precipitation_intensity     78 non-null     float64
 3   mean_precipitation_intensity    78 non-null     float64
 4   max_precipitation_intensity     78 non-null     float64
 5   median_precipitation_intensity  78 non-null     float64
 6   total_cloud_cover               78 non-null     int64  
 7   low_level_cloud_cover           78 non-null     int64  
 8   medium_level_cloud_cover        78 non-null     int64  
 9   high_level_cloud_cover          78 non-null     int64  
 10  temperature                     78 non-null     float64
 11  msl                             78 non-null     f

In [132]:
met_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 88 entries, 2024-08-19 21:00:00+00:00 to 2024-08-29 18:00:00+00:00
Data columns (total 6 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   data.instant.details.air_pressure_at_sea_level  88 non-null     float64
 1   data.instant.details.air_temperature            88 non-null     float64
 2   data.instant.details.cloud_area_fraction        88 non-null     float64
 3   data.instant.details.relative_humidity          88 non-null     float64
 4   data.instant.details.wind_from_direction        88 non-null     float64
 5   data.instant.details.wind_speed                 88 non-null     float64
dtypes: float64(6)
memory usage: 4.8 KB
